In [122]:
import os

In [123]:
%cd "F:\Text-Summarizer"

F:\Text-Summarizer


In [124]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [125]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [126]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [127]:
import os
import urllib.request as request
import zipfile
import mimetypes
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [128]:


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        Function returns None
         """
        unzip_path = self.config.unzip_dir
        logger.info(unzip_path)
        os.makedirs(unzip_path, exist_ok=True)

        # Check if the file is a zip file before extracting
        mime_type, _ = mimetypes.guess_type(self.config.local_data_file)
        if mime_type not in ['application/zip', 'application/x-zip-compressed']:
            logger.error(f"Downloaded file is not a zip file, got: {mime_type}")
        return

        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile:
            logger.error("The file is not a valid zip file or is corrupted.")


In [129]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-06 14:00:04,486: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-06 14:00:04,490: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-06 14:00:04,491: INFO: common: created directory at: artifacts]
[2024-10-06 14:00:04,492: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-06 14:00:04,494: INFO: 1840292391: File already exists of size: ~ 286 KB]
[2024-10-06 14:00:04,495: INFO: 1840292391: artifacts/data_ingestion]
